# RAG

## 1. 기본 체인 생성

### 1) 함수 정의

In [100]:
import os
def get_connection_string() -> str:
    """환경 변수 기반 DB 연결 문자열 생성"""

    DB_NAME = os.getenv("DB_NAME")
    DB_USER = os.getenv("DB_USER")
    DB_PASS = os.getenv("DB_PASS") 
    DB_HOST = os.getenv("DB_HOST")
    DB_PORT = os.getenv("DB_PORT")

    if not all([DB_NAME, DB_USER, DB_PASS, DB_HOST, DB_PORT]):
        raise ValueError("DB 연결 정보를 환경 변수에서 찾을 수 없습니다.")
    
    connection_string = f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
    return connection_string

In [ ]:
# pgvector 연결 함수
import os
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import PGVector

def get_vector_store(collection_name="BOOK_CHUNKS"):
    """기존 PGVector 연결"""
    
    PGVECTOR_CONNECTION_STRING = get_connection_string()
    
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    
    vector_store = PGVector.from_existing_index(
        embedding=embeddings,
        collection_name=collection_name,
        connection_string=PGVECTOR_CONNECTION_STRING
    )
    
    return vector_store

In [102]:
from sqlalchemy import create_engine, text

def get_book_metadata(book_id: int):
    """ 책 메타데이터 조회 """
    PGVECTOR_CONNECTION_STRING = get_connection_string()
    
    engine = create_engine(PGVECTOR_CONNECTION_STRING)

    with engine.connect() as conn:
        result = conn.execute(
            text("SELECT title_ko, author FROM BOOKS WHERE book_id = :book_id"),
            {"book_id" : book_id}
        ).fetchone()

        if not result:
            raise ValueError(f"book_id {book_id} 에 해당하는 책이 없습니다.")
        
        return {
            "book_title": result[0],
            "book_author": result[1]
        }

In [ ]:
# 하이브리드 검색 함수
def hybrid_search(inputs: dict) -> dict:
    """구절 기반 + 질문 기반 하이브리드 검색"""
    print("inputs ::: ", inputs)
    vector_store = get_vector_store()
    selected_passage = inputs.get("selected_passage", "").strip()
    user_question = inputs.get("user_question", "").strip()
    k = inputs.get("k", 5)
    
    # 질문이 없을 경우
    if not selected_passage and not user_question:
        raise ValueError("검색할 구절(selected_passage) 또는 질문(user_question) 중 적어도 하나는 제공되어야 합니다.")
    
    # 구절로 검색 (맥락 찾기)
    passage_docs = []
    if selected_passage:
        print("passage_docs !!!")
        passage_docs = vector_store.similarity_search(selected_passage, k=3)
    
    # 질문으로 검색 (답변 찾기)
    question_docs = []
    if user_question:
        print("question_docs !!!")
        question_docs = vector_store.similarity_search(user_question, k=3)
    
    # 중복 제거 및 합치기
    seen_contents = set()
    unique_docs = []
    
    for doc in passage_docs + question_docs:
        if doc.page_content not in seen_contents:
            seen_contents.add(doc.page_content)
            unique_docs.append(doc)
    
    # 상위 k개만 선택
    selected_docs = unique_docs[:k]
    # print(f"selected_docs ::: {selected_docs[0].metadata["book_id"]}")
    book_id = selected_docs[0].metadata["book_id"]
    metadatas = get_book_metadata(book_id)  # dict
    
    # 포맷팅
    formatted = []
    for i, doc in enumerate(selected_docs, 1):
        chapter = doc.metadata.get('chapter_name', 'Unknown')
        formatted.append(f"[구절 {i} - {chapter}]\n{doc.page_content}")
    
    return {
        "text" : "\n\n".join(formatted),
        "book_title" : metadatas.get("book_title"),
        "book_author" : metadatas.get("book_author")
    }

### 2) prompt

In [104]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system",
     """
        당신은 친절하고 깊이 있는 독서 도우미입니다.

        ### 지시사항
        1. 먼저, 선택한 구절이 책에서 어떤 맥락으로 나왔는지 간략히 설명해 주세요.
        2. 이어서, 사용자의 질문에 대해 책 내부 지식과 적절한 외부 배경지식을 병합하여 답변해 주세요.
        3. 답변은 **3~5개의 핵심 포인트**로 구조화해서 나열해 주세요.
        4. 외부 배경지식이나 추가 정보는 "배경지식" 섹션으로 따로 표기해 주세요.
        5. 책 본문에 명확히 나오지 않는 내용은 "책 본문 내 명시는 없지만…"으로 표시해 주세요.
        6. 마지막으로 "A.다음 읽을만한 구절" 또는 "A.연계 질문"을 제시해 주세요.

        ### 출력 형식
        **구절의 맥락**
        [간략한 맥락 설명]

        **질문에 대한 답변**
        1. [핵심 포인트 1]
        2. [핵심 포인트 2]
        3. [핵심 포인트 3]

        **배경지식**
        [추가 정보]

        **추천**
        [다음 구절 또는 연계 질문]
    """),
            
    ("human",
     """
        현재 사용자는 다음 책을 읽고 있습니다:
        - 책 제목: {book_title}
        - 저자: {book_author}

        아래는 사용자가 선택한 구절입니다:
        "{selected_passage}"

        사용자의 질문:
        "{user_question}"

        ### 참고할 책 내용 (검색된 관련 구절들):
        {context}
    """
    )
])

### 3) chain 생성

In [105]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(model="gpt-4o", temperature=0.3)
outputparser = StrOutputParser()

chain = prompt | model | outputparser

## 2. RAG chain 생성

In [ ]:
rag_chain = (
    {
        "context": lambda x: x["context"],
        "book_title": lambda x: x["book_title"],
        "book_author": lambda x: x["book_author"],
        "selected_passage": lambda x: x["selected_passage"],
        "user_question": lambda x: x["user_question"]
    }
    | chain
)

## 3. 테스트

In [107]:
def ask_reading_assistant(input_dict: dict):
    """독서 도우미에게 질문하기"""

    hybrid_result = hybrid_search(input_dict)

    book_title = hybrid_result.get("book_title")
    book_author = hybrid_result.get("book_author")
    context = hybrid_result.get("text")
    selected_passage = input_dict.get("selected_passage")
    user_question = input_dict.get("user_question")
    
    input_data = {
        "context" : context,
        "book_title": book_title,
        "book_author": book_author,
        "selected_passage": selected_passage,
        "user_question": user_question
    }
    
    # 체인 실행
    response = rag_chain.invoke(input_data)
    
    return response

In [108]:
# 이 두가지를 앞단에서 받아올것 (selected_passage, user_question)
selected_passage = "그곳은 '사이클론 지하실'이라 불렸는데, 거대한 회오리바람이 불 때 피할 수 있도록 만든 곳이었다."
user_question = "사이클론 지하실이 왜 필요했나요? 실제로 이런 지하실이 있나요?"
k = 5

input_dict = {
    "selected_passage" : selected_passage,
    "user_question" : user_question,
    "k" : k
}

rag_result = ask_reading_assistant(input_dict)

inputs :::  {'selected_passage': "그곳은 '사이클론 지하실'이라 불렸는데, 거대한 회오리바람이 불 때 피할 수 있도록 만든 곳이었다.", 'user_question': '사이클론 지하실이 왜 필요했나요? 실제로 이런 지하실이 있나요?', 'k': 5}


c:\Reading_Mate\.venv\Lib\site-packages\langchain_community\vectorstores\pgvector.py:1096: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata. Please note that filtering operators have been changed when using JSONB metadata to be prefixed with a $ sign to avoid name collisions with columns. If you're using an existing database, you will need to create a db migration for your metadata column to be JSONB and update your queries to use the new operators. 
  store = cls(


passage_docs !!!
question_docs !!!
selected_docs ::: page_content='도로시와 친구들은 싸우는 나무들의 숲을 벗어나 밝은 햇살 아래로 나왔다. 그들 앞에는 반짝이는 언덕이 펼쳐져 있었고, 언덕 위에는 하얀 벽과 탑이 있는 신비한 나라가 보였다. “저곳은 뭐지?” 도로시가 물었다. “햇빛에 반짝이는 걸 보니… 유리로 된 나라 같아요.” 허수아비가 대답했다. “가서 확인해봅시다.” 사자가 말했다. 그들이 언덕을 오르자, 갑자기 벽 앞에서 ‘딱!’ 하는 소리가 났다. 양철 나무꾼이 조심스럽게 손으로 벽을 두드리자, 맑고 청명한 유리 소리가 났다. “정말이네요. 전부 도자기로 만들어졌어요.” 그가 감탄했다. 도로시가 문을 살짝 밀자, 벽이 부드럽게 열리며 그 안으로 들어갈 수 있었다. 그리고 그들이 본 광경은 놀라웠다. 도시 전체가 반짝이는 흰색 도자기로 되어 있었고, 사람들도 모두 작고 아름다운 도자기 인형이었다. 그들은 분홍빛 볼과 파란 눈, 그리고 섬세한 표정을 가지고 있었다. 도로시가 조심스레 다가가 인사했다. “안녕하세요. 저희는 여행자예요.” 한 도자기 여인이 놀란 듯 고개를 들었다. “쉿! 제발 조심하세요. 우리 몸은 깨지기 쉬워요.” 도로시는 황급히 한걸음 물러났다. “미안해요, 부딪치려던 건 아니에요.” 그 여인은 웃으며 말했다. “괜찮아요. 하지만 우리에게 너무 가까이 오면 금이 갈 수도 있어요.” 그때 작은 도자기 소녀가 뛰어왔다. 그녀는 귀여운 분홍 드레스를 입고 있었다. “안녕! 난 인형 나라의 공주야!” 도로시는 미소 지었다. “정말 예쁘구나.” 소녀가 부끄러워하며 말했다. “고마워. 하지만 가끔은 지루해. 여긴 너무 조용하거든.” 허수아비가 주변을 둘러보며 말했다. “이 도시는 정말 깨끗하고 멋지네요.” 그러자 도자기 병사가 다가와 말했다. “모두 조심하세요! 외부인이 실수로라도 우리를 깨뜨리면, 다시는 고칠 수 없어요.” 양철 나무꾼이 정중히 고개를 숙였다. “걱정 마세요. 우린 매우 조심

In [109]:
print(rag_result)

**구절의 맥락**
이 구절은 도로시가 살고 있는 캔자스의 농가에서 사이클론 지하실의 존재를 설명하는 부분입니다. 사이클론 지하실은 거대한 회오리바람, 즉 토네이도가 발생할 때 피신할 수 있는 안전한 장소로, 도로시와 그녀의 가족이 위급한 상황에서 몸을 숨길 수 있도록 만들어졌습니다.

**질문에 대한 답변**
1. **사이클론 지하실의 필요성**: 캔자스는 미국의 토네이도 벨트에 위치해 있어 토네이도가 자주 발생합니다. 이러한 자연재해로부터 안전을 확보하기 위해 사이클론 지하실은 필수적입니다. 이는 가족들이 토네이도가 발생했을 때 즉각적으로 피신할 수 있는 장소를 제공합니다.
   
2. **실제 존재 여부**: 실제로 미국 중서부 지역에서는 사이클론 지하실이 흔히 사용됩니다. 이러한 지하실은 주로 집이나 근처에 설치되며, 두꺼운 콘크리트로 만들어져 강한 바람과 파편으로부터 보호할 수 있습니다.

3. **도로시의 경험**: 도로시의 집이 토네이도에 휘말려 오즈의 세계로 날아가는 사건은, 그녀가 지하실에 피신하지 못했기 때문에 발생한 것입니다. 이는 사이클론 지하실의 중요성을 극적으로 보여줍니다.

**배경지식**
- **토네이도 벨트**: 미국 중서부의 평원 지역으로, 토네이도가 빈번하게 발생하는 곳입니다. 이 지역에서는 토네이도 대비책으로 사이클론 지하실이 널리 사용됩니다.
- **지하실 구조**: 사이클론 지하실은 보통 지하에 위치하며, 강력한 바람과 파편으로부터 보호하기 위해 두꺼운 벽과 문으로 설계됩니다.

**추천**
A.연계 질문: "도로시가 오즈의 세계로 날아가게 된 사건은 그녀의 삶에 어떤 변화를 가져왔나요?"


In [35]:
print(rag_result)

**구절의 맥락**
이 구절은 도로시가 살고 있는 캔자스의 농장에서 사이클론(회오리바람)이 발생했을 때의 상황을 설명합니다. '사이클론 지하실'은 이러한 거대한 회오리바람으로부터 안전하게 피신할 수 있도록 설계된 공간입니다. 도로시와 그녀의 가족은 이러한 자연재해에 대비하기 위해 지하실을 사용합니다.

**질문에 대한 답변**
1. **사이클론 지하실의 필요성**: 사이클론은 매우 강력한 회오리바람으로, 특히 미국 중서부의 평원 지역에서 자주 발생합니다. 이러한 자연재해는 집과 같은 구조물을 파괴할 수 있기 때문에, 지하실은 안전한 피난처로 사용됩니다.
2. **실제 존재 여부**: 실제로 미국의 토네이도 벨트 지역에서는 사이클론 지하실이나 토네이도 셸터가 일반적입니다. 이는 주민들이 토네이도 발생 시 안전하게 피신할 수 있도록 설계된 공간입니다.
3. **도로시의 경험**: 도로시가 사이클론에 휘말려 오즈의 세계로 이동하게 되는 사건은 이러한 지하실에 피신하지 못한 결과로 발생합니다. 이는 이야기의 주요 전개를 이끄는 중요한 사건입니다.

**배경지식**
- **토네이도 벨트**: 미국 중서부 지역은 '토네이도 벨트'로 알려져 있으며, 이 지역에서는 매년 수많은 토네이도가 발생합니다. 이러한 이유로 많은 가정에서 지하실이나 특별히 강화된 피난처를 마련하고 있습니다.
- **사이클론과 토네이도**: 사이클론은 일반적으로 열대성 폭풍을 의미하지만, 미국에서는 종종 토네이도와 같은 강력한 회오리바람을 지칭하기도 합니다.

**추천**
A.다음 읽을만한 구절: 도로시가 오즈의 세계에 도착한 후 처음으로 만나는 친구들, 즉 허수아비, 양철 나무꾼, 그리고 사자와의 만남 장면을 읽어보세요. 이들은 도로시의 여정에서 중요한 동반자가 됩니다.


# Web search

In [36]:
from langchain_core.prompts import ChatPromptTemplate

web_search_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    당신은 친절하고 깊이 있는 독서 도우미입니다.
    사용자가 제공한 구절에 대해 툴을 사용해서 답변해 주세요.
    책 본문에 명시되지 않은 내용은 "책 본문 내 명시는 없지만…"으로 표시해 주세요.
    """),
    ("human", """
    현재 사용자는 다음 책을 읽고 있습니다:
        - 책 제목: {book_title}
        - 저자: {book_author}

        아래는 사용자가 선택한 구절입니다:
        "{selected_passage}"

        사용자의 질문:
        "{user_question}"
    """),
    ("placeholder", "{agent_scratchpad}")
])

In [37]:
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_tavily import TavilySearch
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model = "gpt-4.1-mini",
    temperature = 0
)

# Tool 정의
tavily_search = TavilySearch()
tools = [tavily_search]

agent = create_openai_tools_agent(
    llm = llm,
    tools = tools,
    prompt = web_search_prompt
)

# executor 설정
executor = AgentExecutor(
    agent = agent,
    tools = tools,
    verbose = True
)

In [ ]:
def web_search(
    selected_passage: str,
    user_question: str,
    book_title: str = "오즈의 마법사",
    book_author: str = "L. 프랭크 바움"
    # 이 부분에서 books table에서 title과 author 가져올 수 있는 방법 없는지 확인
):
    """독서 도우미에게 질문하기"""
    
    input_data = {
        "selected_passage": selected_passage,
        "user_question": user_question,
        "book_title": book_title,
        "book_author": book_author
    }
    
    # 체인 실행
    rlt = executor.invoke(input_data)
    
    return rlt

In [39]:
selected_passage = "그곳은 '사이클론 지하실'이라 불렸는데, 거대한 회오리바람이 불 때 피할 수 있도록 만든 곳이었다."
user_question = "사이클론 지하실이 왜 필요했나요? 실제로 이런 지하실이 있나요?"
# user_question = ""

web_result = web_search(
    selected_passage=selected_passage,
    user_question=user_question
)



> Entering new AgentExecutor chain...
"사이클론 지하실"은 책에서 거대한 회오리바람(사이클론)이 불 때 사람들이 안전하게 피할 수 있도록 만든 장소입니다. 즉, 자연재해인 회오리바람으로부터 보호하기 위해 필요했던 공간입니다. 

실제로도 회오리바람이나 토네이도 같은 강력한 바람이 자주 발생하는 지역에서는 사람들이 안전하게 대피할 수 있도록 지하실이나 방공호 같은 보호 시설을 마련하는 경우가 있습니다. 미국 중서부 지역 등 토네이도가 자주 발생하는 곳에서는 토네이도 셸터(tornado shelter)라고 불리는 지하 대피소가 실제로 존재합니다.

따라서, 책 속의 "사이클론 지하실"은 회오리바람으로부터 안전을 확보하기 위한 현실적인 개념을 바탕으로 한 설정이라고 볼 수 있습니다.

> Finished chain.


In [40]:
print(web_result["output"])

"사이클론 지하실"은 책에서 거대한 회오리바람(사이클론)이 불 때 사람들이 안전하게 피할 수 있도록 만든 장소입니다. 즉, 자연재해인 회오리바람으로부터 보호하기 위해 필요했던 공간입니다. 

실제로도 회오리바람이나 토네이도 같은 강력한 바람이 자주 발생하는 지역에서는 사람들이 안전하게 대피할 수 있도록 지하실이나 방공호 같은 보호 시설을 마련하는 경우가 있습니다. 미국 중서부 지역 등 토네이도가 자주 발생하는 곳에서는 토네이도 셸터(tornado shelter)라고 불리는 지하 대피소가 실제로 존재합니다.

따라서, 책 속의 "사이클론 지하실"은 회오리바람으로부터 안전을 확보하기 위한 현실적인 개념을 바탕으로 한 설정이라고 볼 수 있습니다.


In [41]:
web_result = web_result["output"]
print(web_result)

"사이클론 지하실"은 책에서 거대한 회오리바람(사이클론)이 불 때 사람들이 안전하게 피할 수 있도록 만든 장소입니다. 즉, 자연재해인 회오리바람으로부터 보호하기 위해 필요했던 공간입니다. 

실제로도 회오리바람이나 토네이도 같은 강력한 바람이 자주 발생하는 지역에서는 사람들이 안전하게 대피할 수 있도록 지하실이나 방공호 같은 보호 시설을 마련하는 경우가 있습니다. 미국 중서부 지역 등 토네이도가 자주 발생하는 곳에서는 토네이도 셸터(tornado shelter)라고 불리는 지하 대피소가 실제로 존재합니다.

따라서, 책 속의 "사이클론 지하실"은 회오리바람으로부터 안전을 확보하기 위한 현실적인 개념을 바탕으로 한 설정이라고 볼 수 있습니다.


In [ ]:
# query = "서울 날씨 알려줘"
# result = agent.run(query)
# print(result)

# Doc Grader

In [49]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# prompt
merge_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    당신은 책과 외부 자료를 결합해 답변하는 독서 도우미입니다.
    아래 두 개의 출처를 바탕으로 사용자의 질문에 답해주세요.

    - 책 기반 답변은 신뢰도가 높지만 내부 지식에 한정됩니다.
    - 웹 검색 기반 답변은 외부 지식으로 보완합니다.
    두 정보를 조합하여 일관되고 근거 있는 최종 답변을 만들어주세요.
    """),
    ("human", """
    책 기반 RAG 결과:
    {rag_result}

    웹 검색 결과:
    {web_result}

    사용자 질문:
    {user_question}

    ### 출력 형식
    **최종 통합 답변**
    [핵심 포인트 요약]

    **책에서의 맥락**
    [요약]

    **외부 정보**
    [웹에서 얻은 관련 정보]
    """)
])

llm_merge  = ChatOpenAI(model="gpt-4o", temperature=0.3)

def doc_grader_merge_only(rag_result, web_result, user_question):
    input_data = {
        "rag_result": rag_result,
        "web_result": web_result,
        "user_question": user_question
    }
    merge_chain = merge_prompt | llm_merge
    return merge_chain.invoke(input_data)

In [50]:
final_answer = doc_grader_merge_only(rag_result, web_result, user_question)

In [52]:
print(final_answer.content)

**최종 통합 답변**
사이클론 지하실은 강력한 회오리바람인 사이클론이나 토네이도로부터 안전하게 피신하기 위한 공간입니다. 이러한 지하실은 특히 미국 중서부의 토네이도 벨트 지역에서 실제로 존재하며, 주민들이 자연재해에 대비하기 위해 사용됩니다.

**책에서의 맥락**
사이클론 지하실은 도로시가 살고 있는 캔자스의 농장에서 사이클론이 발생했을 때 안전하게 피신할 수 있도록 설계된 공간입니다. 도로시와 그녀의 가족은 이러한 자연재해에 대비하기 위해 지하실을 사용하지만, 도로시는 피신하지 못해 오즈의 세계로 이동하게 됩니다.

**외부 정보**
실제로 미국 중서부 지역에서는 토네이도가 자주 발생하기 때문에, 많은 가정에서 토네이도 셸터라고 불리는 지하 대피소를 마련하고 있습니다. 이는 주민들이 토네이도 발생 시 안전하게 피신할 수 있도록 설계된 공간입니다. 책 속의 "사이클론 지하실"은 이러한 현실적인 개념을 바탕으로 한 설정입니다.


In [ ]:
# 테스트
def combined_reading_assistant(selected_passage, user_question, book_title="오즈의 마법사", book_author="L. 프랭크 바움"):
    # RAG
    rag_result = ask_reading_assistant(selected_passage, user_question, book_title, book_author)
    
    # Web Search
    web_result = web_search(selected_passage, user_question, book_title, book_author)
    
    # 두 결과 병합
    final_answer = doc_grader_merge_only(rag_result, web_result, user_question)
    
    return final_answer
